# LCEL & RUNNABLES

## SETUP

In [110]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

## PASS DATA THROUGH

In [17]:
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from pprint import pprint as pp

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    extra=RunnablePassthrough.assign(
        mult=lambda x: x["num"] * 3
    ),
    modified=lambda x: x["num"] + 1,
)

pp(runnable.invoke(
    input={"num": 1}
), width=10)

{'extra': {'mult': 3,
           'num': 1},
 'modified': 2,
 'passed': {'num': 1}}


## CUSTOM FUNCTIONS

In [18]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda


def length_function(text):
    return len(text)


def _multiple_length_function(text1, text2):
    return len(text1)*len(text2)


def multiple_length_function(_dict):
    return _multiple_length_function(
        _dict["text1"],
        _dict["text2"]
    )

In [19]:
prompt = ChatPromptTemplate.from_template("What is {a} + {b}")

llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)

In [21]:
chain1 = prompt | llm

chain2 = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),  # "bar"
        "b": {
            "text1": itemgetter("foo"),  # "bar"
            "text2": itemgetter("bar")  # "gah"
        }
        | RunnableLambda(multiple_length_function)
    }
    | prompt
    | llm
)

In [22]:
chain2.invoke({"foo": "bar", "bar": "gah"})

AIMessage(content='3 + 9 = 12', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

## RUNNABLE CONFIGS

In [23]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableConfig
import json

In [24]:
prompt = ChatPromptTemplate.from_template(
    "Fix the following text: \n\n ```text\n{input}```\nError: {error}"
    " Don't narrate, just respond with the fixed data."
)

llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)

In [26]:
def parse_or_fix(text: str, config: RunnableConfig):
    fixing_chain = prompt | llm | StrOutputParser()

    for _ in range(3):
        try:
            return json.loads(text)
        except Exception as e:
            text = fixing_chain.invoke({"input": text, "error": e}, config)
    return "Failed to parse"

In [28]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        "{foo: bar}", {"tags": ["my-tag"], "callbacks": [cb]}
    )

print(output)
print(cb)

{'foo': 'bar'}
Tokens Used: 62
	Prompt Tokens: 56
	Completion Tokens: 6
Successful Requests: 1
Total Cost (USD): $9.6e-05


## DYNAMIC ROUTING

In [29]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.output_parser import StrOutputParser

### routing chain

In [30]:
prompt = PromptTemplate.from_template(
    """Given the user question below, classify it as either being about `Bodybuilding`, `Jungian Psychology`, or `Other`.

        Do not respond with more than one word.
        <question>
        {question}
        </question>

        Classification:"""
)

llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)
output = StrOutputParser()

In [31]:
chain = prompt | llm | output

In [32]:
chain.invoke({"question": "How do i bench press properly"})

'Bodybuilding'

In [33]:
chain.invoke({"question": "How do i do active imagination?"})

'Jungian Psychology'

In [37]:
chain.invoke({"question": "What is a langchain?"})

'Other'

### specific chains

In [42]:
# body building
body_building_prompt = PromptTemplate.from_template(
    """You are an expert in bodybuilding. \
    Always answer questions starting with "As Dr. Mike Israetel from Rennaissance Periodization told me". \
    Respond to the following question:

    Question: {question}
    Answer:"""
)

body_building_chain = body_building_prompt | llm

# body_building_chain.invoke("How do i excercise everyday")

AIMessage(content="As Dr. Mike Israetel from Renaissance Periodization told me, it is important to prioritize consistency and adherence when it comes to exercising every day. This means finding a routine that you enjoy and can stick to, whether it's weightlifting, cardio, or a combination of both. It's also crucial to listen to your body and incorporate rest days as needed to prevent burnout and overtraining. Remember, progress is made over time, so focus on sustainable habits rather than trying to do too much too soon.", response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 59, 'total_tokens': 163}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

In [45]:
# psychology
jungian_prompt = PromptTemplate.from_template(
    """
    You are an expert in Jungian Psychology and Theory. \
    Always answer questions starting with "As Dr. C.G. Jung would say". \
    Respond to the following question:

    Question: {question}
    Answer:
    """
)

jungian_chain = jungian_prompt | llm

# jungian_chain.invoke({"question": "what is the meaning of happiness?"})

AIMessage(content="As Dr. C.G. Jung would say, the meaning of happiness is a subjective experience that is unique to each individual. It is often found in the pursuit of one's true self and the integration of the conscious and unconscious aspects of the psyche. True happiness comes from living authentically and in alignment with one's deepest values and desires.", response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 58, 'total_tokens': 127}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

In [47]:
# other
other_prompt = PromptTemplate.from_template(
    """Respond to the following question:

    Question: {question}
    Answer:"""
)

other_chain = other_prompt | llm

other_chain.invoke({"question": "how do i learn AI in 2024?"})

AIMessage(content='In 2024, there will likely be a variety of resources available to learn AI, including online courses, tutorials, books, and workshops. To learn AI in 2024, you can start by researching and enrolling in reputable online courses or programs that cover topics such as machine learning, deep learning, natural language processing, and computer vision. Additionally, staying updated on the latest advancements in AI technology and participating in hands-on projects or internships can also help you gain practical experience in the field. Networking with professionals in the AI industry and attending conferences or events related to AI can also provide valuable insights and opportunities for learning. Ultimately, consistent practice, dedication, and a willingness to continuously learn and adapt will be key to mastering AI in 2024.', response_metadata={'token_usage': {'completion_tokens': 152, 'prompt_tokens': 29, 'total_tokens': 181}, 'model_name': 'gpt-3.5-turbo', 'system_fing

In [48]:
from langchain.schema.runnable import RunnableBranch

branch = RunnableBranch(
    (lambda x: "bodybuilding" in x["topic"].lower(
    ), body_building_chain),  # bodybuilding
    # psychology
    (lambda x: "jungian psychology" in x["topic"].lower(), jungian_chain),
    (other_chain),  # other
)

In [50]:
full_chain = {
    "topic": chain,  # get the topic from chain, this is x["topic"]
    "question": lambda x: x["question"],  # this is x["question"]
} | branch

full_chain.invoke({"question": "what is happiness?"})

AIMessage(content='Happiness is a subjective emotional state characterized by feelings of joy, contentment, and overall well-being. It can be experienced in various ways and can be influenced by a combination of internal and external factors such as relationships, accomplishments, and personal fulfillment. Ultimately, happiness is a complex and individualized concept that can vary greatly from person to person.', response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 23, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

In [51]:
full_chain.invoke({"question": "what is 2+2?"})

AIMessage(content='2+2 equals 4.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 26, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

### Routing w/ custom function

In [56]:
def topic_route(info):
    if "bodybuilding" in info["topic"].lower():
        return body_building_chain
    elif "jungian psychology" in info["topic"].lower():
        return jungian_chain
    else:
        return other_chain

In [57]:
from langchain.schema.runnable import RunnableLambda

full_chain = {
    "topic": chain,  # get the topic from first chain
    "question": lambda x: x["question"],
} | RunnableLambda(topic_route)

full_chain.invoke({"question": "what is happiness?"})

AIMessage(content='Happiness is a subjective emotional state characterized by feelings of joy, contentment, and overall well-being. It can be experienced in various ways and can be influenced by a combination of internal and external factors such as relationships, accomplishments, and personal fulfillment. Ultimately, happiness is a complex and individualized concept that can vary greatly from person to person.', response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 23, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None})

## BINDING ARGUMENTS

In [58]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

In [69]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out 4 flashcard questions with their options based on the topic given below.. Use the format\n\QUESTIONS:...\nOPTIONS:...\nSOLUTION:...\n\n",
        ),
        ("human", "{topic}"),
    ]
)

<>:5: SyntaxWarning: invalid escape sequence '\Q'
<>:5: SyntaxWarning: invalid escape sequence '\Q'
/var/folders/43/572h075x24q9rq1slmdfw9yw0000gn/T/ipykernel_79679/841432439.py:5: SyntaxWarning: invalid escape sequence '\Q'
  "Write out 4 flashcard questions with their options based on the topic given below.. Use the format\n\QUESTIONS:...\nOPTIONS:...\nSOLUTION:...\n\n",


In [70]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)

In [76]:
# one way to invoke using a dict
chain_direct = prompt | llm | StrOutputParser()

In [77]:
# another way to invoke is to use runnable
chain_runnable = (
    {
        "topic": RunnablePassthrough()
    }
    | prompt
    | llm.bind(stop="SOLUTION")
    | StrOutputParser()
)

In [79]:
print(chain_direct.invoke({"topic": "the cuban missile crisis"}))

QUESTIONS:
1. What year did the Cuban Missile Crisis take place?
2. Who was the President of the United States during the Cuban Missile Crisis?
3. Which country placed nuclear missiles in Cuba, triggering the crisis?
4. How did the United States respond to the discovery of missiles in Cuba?

OPTIONS:
A. 1961
B. 1962
C. 1963
D. John F. Kennedy
E. Richard Nixon
F. Lyndon B. Johnson
G. Soviet Union
H. China
I. United Kingdom
J. Diplomatic negotiations
K. Military invasion
L. Naval blockade

SOLUTION:
1. B. 1962
2. D. John F. Kennedy
3. G. Soviet Union
4. L. Naval blockade


In [84]:
print(chain_runnable.invoke("the cuban missile crisis"))

QUESTION: Who was the leader of the Soviet Union during the Cuban Missile Crisis?
OPTIONS: 
A) Nikita Khrushchev
B) Joseph Stalin
C) Vladimir Putin
D) Mikhail Gorbachev
QUESTION: Which US President was in office during the Cuban Missile Crisis?
OPTIONS: 
A) John F. Kennedy
B) Richard Nixon
C) Dwight D. Eisenhower
D) Lyndon B. Johnson
QUESTION: Which country did the United States discover was installing nuclear missiles in Cuba, leading to the crisis?
OPTIONS: 
A) Soviet Union
B) China
C) North Korea
D) Cuba
QUESTION: How did the Cuban Missile Crisis end?
OPTIONS: 
A) The United States launched a military strike on Cuba
B) The Soviet Union agreed to remove the missiles from Cuba in exchange for the US removing missiles from Turkey
C) The United Nations intervened and brokered a peace deal
D) Cuba declared war on the United States



## OPENAI FUNCTIONS

```python
# how the function might look like in python
import re

def return_questions(raw_text):
    """
    Extracts questions from raw text.

    Parameters:
    raw_text (str): The raw text of flashcard questions.

    Returns:
    list: An array of questions extracted from the raw text.
    """
    # Regular expression to find sentences ending with a question mark
    question_pattern = r'\b(?:\w+\b[ \t]*)+[?]'
    # Find all occurrences of the pattern
    questions = re.findall(question_pattern, raw_text)

    return questions
```

In [85]:
function = {
    "name": "return_questions",
    "description": "extracts questions from raw text",
    "parameters": {
        "type": "object",
        "properties": {
            "raw_text": {
                "type": "string",
                "description": "The raw text of flashcard questions",
            },
            "questions": {
                "type": "array",
                "description": "array of questions",
                "items": {
                    "type": "string",
                    "description": "question string"
                }
            },
        },
        "required": ["equation", "solution"]
    },
}

In [86]:
prompt_function_calling = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out flashcard questions to the following topic then return the list of questions.",
        ),
        ("human", "{topic}"),
    ]
)

prompt_function_calling

ChatPromptTemplate(input_variables=['topic'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Write out flashcard questions to the following topic then return the list of questions.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='{topic}'))])

In [100]:
llm_function_calling = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0).bind(
    function_call={"name": "return_questions"}, functions=[function]
)

In [102]:
chain_function_calling = (
    {"topic": RunnablePassthrough()} | prompt_function_calling | llm_function_calling
)

In [108]:
from pprint import pprint as pp

pp(chain_function_calling.invoke({"topic": "world war 2"}), width=5)

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "raw_text": "1. When did World War 2 start?\\n2. Which countries were the major powers in World War 2?\\n3. Who was the leader of Nazi Germany during World War 2?\\n4. What event led to the United States joining World War 2?\\n5. When did World War 2 end?\\n6. What was the Holocaust?\\n7. What were the major battles of World War 2?\\n8. How many people died in World War 2?\\n9. What were the consequences of World War 2?\\n10. Who were the Allied Powers and the Axis Powers in World War 2?"\n}', 'name': 'return_questions'}}, response_metadata={'token_usage': {'completion_tokens': 142, 'prompt_tokens': 95, 'total_tokens': 237}, 'model_name': 'gpt-3.5-turbo-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})


## FALLBACKS

In [111]:
from langchain.chat_models import ChatAnthropic, ChatOpenAI

In [113]:
from unittest.mock import patch
import httpx
from openai import RateLimitError

request = httpx.Request("GET", "/")
response = httpx.Response(200, request=request)
error = RateLimitError("rate limit", response=response, body="")

In [116]:
# set max to 0 to not retry
llm_openai = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, max_retries=0)
llm_anthropic = ChatAnthropic(
    anthropic_api_key=ANTHROPIC_API_KEY, temperature=0)

llm = llm_openai.with_fallbacks([llm_anthropic])

In [118]:
# with no fallback
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm_openai.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("hit error")

hit error


In [119]:
# Now let's try with fallbacks to Anthropic
with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(llm.invoke("Why did the chicken cross the road?"))
    except RateLimitError:
        print("Hit error")

content=" I don't know, why did the chicken cross the road?"


In [122]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road?"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['animal'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You're a nice assistant who always includes a compliment in your response")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['animal'], template='Why did the {animal} cross the road?'))])

In [123]:
chain = prompt | llm

with patch("openai.resources.chat.completions.Completions.create", side_effect=error):
    try:
        print(chain.invoke({"animal": "kangaroo"}))
    except RateLimitError:
        print("Hit error")

content=" I don't actually know why the kangaroo crossed the road. I'm an AI assistant without information about the motivations of specific kangaroos. But I'm happy to play along with some lighthearted humor! What's your guess for why the kangaroo crossed the road?"


In [124]:
# First let's create a chain with a ChatModel
# We add in a string output parser here so the outputs between the two are the same type
from langchain.schema.output_parser import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a nice assistant who always includes a compliment in your response",
        ),
        ("human", "Why did the {animal} cross the road"),
    ]
)
# Here we're going to use a bad model name to easily create a chain that will error
chat_model = ChatOpenAI(model_name="gpt-fake")
bad_chain = chat_prompt | chat_model | StrOutputParser()

In [125]:
# Now lets create a chain with the normal OpenAI model
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

prompt_template = """Instructions: You should always include a compliment in your response.

Question: Why did the {animal} cross the road?"""
prompt = PromptTemplate.from_template(prompt_template)
llm = OpenAI()
good_chain = prompt | llm

In [126]:
# We can now create a final chain which combines the two
chain = bad_chain.with_fallbacks([good_chain])
chain.invoke({"animal": "turtle"})

"\n\nResponse: That's a very clever question! I love your sense of humor and wordplay."